# Memory project - Database vectorization

## Secrets

In [51]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY= os.getenv('PINECONE_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
STEAMSHIP_API_KEY = os.getenv('STEAMSHIP_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
GEMINI_KEY = os.getenv('GEMINI_KEY')

os.environ['PATH'] += os.pathsep + '/usr/bin'

## Libraries

In [52]:
import os
import json
import pandas as pd
import shutil
import openai
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from pinecone import Index  # Import Index for Pinecone operations
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone

# Install missing packages
# %pip install sentence-transformers

from sentence_transformers import SentenceTransformer


## Family safe 

## Read json + create local df/csv

In [54]:
# V2

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def process_json_files(data_dir=r'.\Family safe', processed_dir=r'.\Processed JSONs', output_file=r'.\combined_data.csv'):
    """
    Processes all JSON files in the specified directory, adds new data to an existing DataFrame, 
    and ensures no duplicates.

    Args:
        data_dir: Directory containing JSON files.
        processed_dir: Directory where processed files will be moved.
        output_file: Filepath for the saved combined DataFrame.

    Returns:
        A DataFrame containing the updated processed data.
    """
    all_data = []

    # Ensure the processed directory exists
    os.makedirs(processed_dir, exist_ok=True)

    # Process each JSON file
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                json_data = json.load(f)

            # Extract document-level metadata
            name = json_data.get("Name", "Unknown")
            doc_type = json_data.get("Type", "Unknown")
            author = json_data.get("Author", "Unknown")
            date = json_data.get("Date", "Unknown")

            # Initialize a page counter
            page_counter = 1

            # Process each page
            for page in json_data.get("Pages", []):
                page_number = page.get("Page Number", None)
                
                # Assign a sequential number if Page Number is NaN
                if page_number is None:
                    page_number = page_counter
                    page_counter += 1  # Increment the counter

                page_data = {
                    "Doc name": name,
                    "Type": doc_type,
                    "Author": author,
                    "Date": date,
                    "Text": page.get("Extracted Text", ""),
                    "Page_number": page_number,
                }
                all_data.append(page_data)


            # Move the processed file
            processed_filepath = os.path.join(processed_dir, filename)
            os.rename(filepath, processed_filepath)

    # Create a DataFrame from new data
    new_df = pd.DataFrame(all_data)

    # Add summaries to the new DataFrame
    new_df = populate_summary_column(new_df)

    # Reorder columns
    new_df = new_df[["Doc name", "Type", "Author", "Date", "Text", "Page_number", "Summary"]]

    # Check if an existing DataFrame exists
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True).drop_duplicates()
    else:
        combined_df = new_df

    # Save the updated DataFrame
    combined_df.to_csv(output_file, index=False)
    print(f"Updated DataFrame saved to {output_file}")

    return combined_df

def populate_summary_column(df):
    """
    Populates the 'Summary' column in the DataFrame using generate_text_summaries.
    
    Args:
        df: The DataFrame containing the text data.

    Returns:
        The DataFrame with the 'Summary' column populated.
    """
    # Extract texts from the DataFrame
    texts = df["Text"].tolist()

    # Generate summaries for the texts
    text_summaries, _ = generate_text_summaries(texts, tables=None, summarize_texts=True)

    # Assign the summaries back to the DataFrame
    df["Summary"] = text_summaries

    return df

def generate_text_summaries(texts, tables=None, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} 
    Do not include "this is a summary" at the begining of the summary.
    The summary must be in French. """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


In [55]:
# Source and target are the same for testing
df=process_json_files(data_dir=r'data\Family safe', processed_dir=r'data\Family safe')

# df=process_json_files(data_dir=r'data\Family safe', processed_dir=r'data\Family safe\Processed JSONs')

Updated DataFrame saved to .\combined_data.csv


In [56]:
# del df
df.head(100)

,Doc name,Type,Author,Date,Text,Page_number,Summary
0,Pdf img,scan,John Doe,2025-01-23 15:43:26,"C'est le dernier, Isaac, dit Hovel, le plus so...",118,"Isaac, souvent appelé Louis, est l'ancêtre dir..."
1,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,POUR LA MÉMOIRE\nFAMILIALE\n\nFAMILLE HISTOIRE...,1,"Mémoires familiales de Jean-Georges Lambert, V..."
2,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,J'ai entrepris ce travail pour mes fils qui ti...,1,L'auteur réalise ce travail pour ses fils et s...
3,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,VOLUME 1\n\nPage\n6- Préambule.\n\nTABLE DES M...,2,VOLUME 1\n\nCe volume comprend un préambule et...
4,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,"-Les sentiments de Bonaparte, puis de Napoléon...",3,Les sentiments de Bonaparte et de Napoléon env...
...,...,...,...,...,...,...,...
92,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,"92\n\nDopulation juive alsacienne en 1825, éta...",92,"En 1825, la population juive en Alsace était e..."
93,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,Un autre aspect de la volonté d'oubli. Au fur ...,93,"Au XIX<sup>e</sup> siècle, les milieux juifs d..."
94,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,94\n\nCet évolution du judaisme dans les espri...,94,L'évolution du judaïsme en Europe occidentale ...
95,Pour la mémoire familiale 1-50,scan,Jean Lambert,2025-01-31 15:10:05,95\n\nautrement que de reconnaître leurs brill...,95,Le texte aborde la contribution des Juifs à la...


In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import nltk
import pandas as pd

# Ensure NLTK tokenizer is available
nltk.download("punkt")

# Define a word-based text splitter with correct overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,  # Max words per chunk
    chunk_overlap=120,  # Overlap words
    separators=[" ", "\n"],  # Ensures split happens at spaces or newlines
    length_function=lambda text: len(nltk.word_tokenize(text)),  # Count words, not characters
)

def split_text_dataframe(df):
    """Splits a DataFrame’s ‘Text’ column into chunks across multiple pages, ensuring overlap."""
    documents = []
    prev_text = ""  # Store last portion of previous page for cross-page overlap

    for i, row in df.iterrows():
        current_text = prev_text + " " + row["Text"] if prev_text else row["Text"]
        
        text_chunks = text_splitter.split_text(current_text)  # Word-based splitting with overlap
        total_chunks = len(text_chunks)
        
        for j, chunk in enumerate(text_chunks):
            # Copy row data and create new chunk
            new_row = row.drop(labels=["word_count"], errors="ignore").copy()
            new_row["Text"] = chunk

            # Ensure Chunk ID uses only integers
            new_row["Chunk_ID"] = f"{row['Doc name']}_Chunk{int(j+1)}"
            new_row["Total_Chunks"] = int(total_chunks)

            # Convert into LangChain Document format
            documents.append(
                Document(
                    page_content=chunk,
                    metadata={
                        "Doc name": row["Doc name"],
                        "Author": row["Author"],
                        "Page_number": row["Page_number"],
                        "Chunk_ID": new_row["Chunk_ID"],
                        "Total_Chunks": new_row["Total_Chunks"],  # Ensure integer
                    }
                )
            )

        # Store the last portion (overlap) of this page to carry forward
        prev_text = " ".join(nltk.word_tokenize(current_text)[-120:])  # Preserve last 120 words

    return documents

# Process DataFrame to Generate Chunks
docs2 = split_text_dataframe(df)

# Print the output to verify chunk overlap
for doc in docs2:
    print(f"Chunk ID: {doc.metadata['Chunk_ID']}, Page: {doc.metadata['Page_number']}\n{doc.page_content}\n")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Chunk ID: Pdf img_Chunk1, Page: 118
C'est le dernier, Isaac, dit Hovel, le plus souvent appelé Louis qui est notre ancêtre direct à la génération suivante. Il est né en 1766 à Froeningen, et s'est marié avec Rachel (ou Reiche ou Rosalie ou Thérèse) Gugenheim, née également à Froeningen en 1772, et décédée dans le même village en 1843, à 81 ans.

Du ménage Louis et Rachel, je connais six enfants: Charlotte ex Judele, Jacques ex Jacob, marchand de bestiaux, Alexandre ex Samuel, marchand de bestiaux, Lehmann ou Clément, Marx, marchand de bétail, et Julie ex Sara, tous nés à Froeningen entre 1795 et 1802.

Parmi eux, notre ancêtre direct est Lehmann, sur son prénom je note qu'en 1808 il devient Clément, mais que seul le prénom de Lehmann continuera à être employé par la suite. Il est né en 1800 à Froeningen, et décédé au même lieu en 1843. En 1827 il a épousé Athelle (ou Adèle, ou Esther) Bloch, née à Soultz, village proche de Froeningen, en 1803, que j'ai déjà mentionnée.

Lehmann et Adèl

## Creating Pinecone DB

In [59]:
from pinecone import Pinecone

In [60]:
import pinecone as pc
from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)

In [61]:
import time

index_name = "memory-project4"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [62]:
# Install missing packages
%pip install langchain

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [63]:
import os
from getpass import getpass
import torch

from langchain.embeddings.openai import OpenAIEmbeddings

# get API key from top-right dropdown on OpenAI website
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass("Enter your OpenAI API key: ")
model_name = 'text-embedding-3-small'

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY,
    # device=device -> Not compatible with OpenAI embeddings
)

In [64]:
# Indexing v1

from langchain.vectorstores import Pinecone as PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    docs2,
    embed,  # Your embedding function
    index_name=index_name
)
